# selenium-显示等待，获取动态网页数据

In [1]:
#【综合练习】：显示等待爬取动态网页

#在人邮出版社主页搜索框输入内容（数据结构）后，爬取所有搜索结果（7页结果）
#爬取所有页面，有时候在页面直接可以获取总页数，用for循环也可以完成
#本题没有总页数，用最后一页上，“下一页”按钮失效来控制循环结束

# wait = WebDriverWait(driver, 10)    # 设置等待时长
# wait.until( EC.presence_of_all_elements_located( (By.XPATH,'...') ) ) 获取所有同类标签的列表，无需看见该元素
# wait.until( EC.presence_of_element_located( (By.XPATH,'...') ) )   只获取第一个指定元素
# wait.until( EC.element_to_be_clickable( (By.XPATH, '...')) )
# wait.until( EC.visibility_of_element_located( (By.XPATH, '...')) )


import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from lxml import etree
import time

driver = webdriver.Chrome()       # 打开chrome浏览器
url='https://www.ptpress.com.cn/'
driver.get(url) #访问指定页面，主页

#用代码向受控chrome发送点击命令，实现网页自动跳转，以下两句等价
#search_btn = driver.find_element_by_css_selector('#header > div.container > div > div.col-md-8.tools > div.search > input[type=text]')

search_btn = driver.find_element(By.XPATH,'//*[@id="header"]/div[1]/div/div[2]/div[3]/input')
search_btn.send_keys('数据结构')    # 将关键字填入搜索框进行搜索
wait = WebDriverWait(driver, 10)    # 设置等待时长

#点击搜索按钮，以下两句等价
#confirm_btn = wait.until( EC.element_to_be_clickable( (By.CSS_SELECTOR, '#header > div.container > div > div.col-md-8.tools > div.search > button')))
confirm_btn = wait.until( EC.element_to_be_clickable( (By.XPATH, '//*[@id="header"]/div[1]/div/div[2]/div[3]/button')))
confirm_btn.click()

#切换到浏览器下一个窗口
# windows =driver.window_handles #windows是一个list类型
driver.switch_to.window(driver.window_handles[-1])  #切换到最新的一个窗口


#点击“更多”按钮
more_btn = wait.until( EC.element_to_be_clickable( (By.XPATH, '//*[@id="booksMore"]')))
more_btn.click()

driver.switch_to.window(driver.window_handles[-1]  )  #再次切换到最新的一个窗口

df=pd.DataFrame(columns=['NAME','LINK'])
# time.sleep(3) #加载时间不够，会导致爬漏
next_btn = wait.until( EC.element_to_be_clickable( (By.XPATH, '//*[@class="ivu-page-next"]')))

while True:
    #将当前页面转为dom结构，并获取单页数据
    
#     dom=etree.HTML(driver.page_source)   #将str转为dom结构
#     list=dom.xpath('//*[@id="books_content"]/div/div/a/p/text()')
#    不用以上静态方式
    books=wait.until( EC.presence_of_all_elements_located( (By.XPATH,'//*[@id="books_content"]/div/div/a/p') ) )
    booknames=[i.text for i in books]
    
    alists=wait.until( EC.presence_of_all_elements_located( (By.XPATH,'//*[@id="books_content"]/div/div/a') ) )
    booklinks=[i.get_attribute('href')  for i in alists]
    
    df1=pd.DataFrame({'NAME':booknames,'LINK':booklinks})
    df=pd.concat([df,df1],ignore_index=True)

    
    if next_btn.get_attribute('class')=="ivu-page-next ivu-page-disabled":
        break
        
    next_btn = wait.until( EC.element_to_be_clickable( (By.XPATH, '//*[@class="ivu-page-next"]')))
    next_btn.click()
    time.sleep(3)  
    #本例以尝试其他很多方法，去掉time_sleep结果均不完整
    #原因是webdriver的显示等待（隐式等待）都不会等到ajax异步加载
    #因此，爬取时，最常采用的手段就是显示等待+time.sleep(1)

    

df.to_csv("d:/aa/temp/0411123.csv",encoding='gbk')

# list( df['NAME'])



In [ ]:
# import selenium
# help(selenium)  #查看版本

In [3]:
#【练习】
#利用selenium获取网页数据,并通过lxml获取网页数据--不重复，不漏爬
#获取新书推荐中，尝试获取计算机、电子、科普等若干类目的 8本图书

#本例用的wait.until(EC.visibility_of_all_elements_located((By.XPATH,...)) 形成列表再循环


from selenium import webdriver
from lxml import etree
import time

#导入相应的类模块及函数
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

driver = webdriver.Chrome()       # 打开chrome浏览器
driver.get('https://www.ptpress.com.cn/') #访问指定页面，主页


,计算机,电子,科普,通信,摄影,经济,管理,金融与投资,成功/励志,心理学,设计,音乐,影视传媒,体育,生活,家教育儿,绘画,少儿
0,Python编程快速上手2：趣味小项目轻松学,运算放大器权威指南（第5版）,考试脑科学2：记忆、压力、动机的脑科学真相,5GtoB字典及面向深度融合的6G演进,故宫秘境,从零到亿,管理的博弈,股票大作手回忆录 丁圣元注疏版（签章版）,新潮职业,津巴多普通心理学（第8版）,商业插画设计思维魔方 形色构成+构图+色彩+光影+造型语言+插画后期+原创思维+综合训练,我的第一套音乐启蒙书 世界名曲篇,Hi-Fi High-End音响设计史,膝关节功能强化训练 预防损伤 缓解慢性疼痛与提升运动表现,新手养猫指导手册 从读懂猫咪心理到温暖相伴一生,日记十招：特级教师高子阳教你轻松写日记,花姬 Hanasa原子Dan双人典藏插画集,丝绸之路历史百科绘本 乘风破浪 开往世界的船
1,人工智能：现代方法（第4版）（上下册）,无线电合订本 66周年版上,视觉之旅：神奇的化学元素（少儿版）,5G+AI融合全景图,造像之术 职业修图师的商业摄影后期精修 实战篇,不公平优势：如何找到阻力最小的成功路径,激活组织能量：打造有机组织，创造非凡业绩,跨境电商亚马逊是如何运营的 从零做到年销千万实操手册,费曼学习法：我就是这样考上清华的,如何拥抱一只刺猬：恋爱与婚姻中的人格识别、接纳与付出,青少年美育趣味课堂 XMind思维导图制作,音乐制作自学手册 Cubase编曲案例分析与实战教学 视频版,什么是故事 给导演和编导的编剧与叙事指南,肩关节功能强化训练 预防损伤缓解慢性疼痛与提升运动表现,细工花饰的创意手作笔记 四时之趣,闯进诗词才子班 秒懂漫画古诗词 第一辑,动漫人体结构表现技法专项训练,丝绸之路历史百科绘本 勇往直前 走向世界的路
2,动手学深度学习（PyTorch版）,无线电合订本 66周年版下,量子矩阵：奇异的量子世界之旅,6G移动通信系统 理论与技术,一本书学会手机短视频拍摄,股权架构设计与股权激励,数智化敏捷组织：云钉一体驱动组织转型,金税四期管控下企业纳税筹划实务指南,你的灯亮着吗：如何找到真问题（10万册纪念版）,悲伤即成长：那些打不倒你的 只会让你更强大,扁平插画绘制专业教程,简谱约翰汤普森简易钢琴教程 五线谱+简谱版,混音指南 第2版,谢军少儿国际象棋启蒙绘本,手鞠球的几何世界 手鞠球制作教程,认知觉醒：伴随一生的学习方法论（青少年学习版）,鬼刀3 苍蓝 WLOP典藏插画作品集,STEAM探索 法布尔昆虫记互动立体书 蜘蛛和蝎子
3,人工智能伦理与治理——未来视角,模拟音频功率放大器设计,美丽的化学元素,御风前行 广电5G融合之路,故事感美食摄影,静水流深：高水平对外投资的实践逻辑,刷新管理：数字化企业快速成长,创新定价：世界知名企业的最大化盈利法则,复盘高手：自我认知与自我精进的底层逻辑,认知行为疗法：会谈技能与咨询现场,品牌设计法则,培生经典音乐教程 音乐理论基础 第三版,影视特效 运动匹配的艺术与技巧,运动营养全书 第5版,簪娘入门指南 古风饰品制作教程,费曼学习法（漫画版）：清华学霸的高效学习手册,鬼刀 WLOP个人插画作品集 典藏修订版,奇思妙想树叶画 一学就会的创意植物手工
4,RISC-V体系结构编程与实践,STM32入门100步,课堂上学不到的数学,自动驾驶电动汽车的计算与通信架构 车载网络通信架构,造像之术 职业修图师的商业摄影后期精修 技法篇,数字政府：开辟国家治理现代化新境界,华为的意志：华为经营逻辑的引擎,一小时漫画基金实战法,跑赢：成功不止一种定义,我会永远抱紧你,Procreate新国风插画绘制技法与商业应用,音乐制作自学手册 Logic Pro编曲案例分析与实战教学 视频版,拍摄手册：77种方法让你的影片更完美,抗阻训练方案设计指南 第4版,超可爱立体书 手工小机关制作入坑指南,适度养育：培养独立且自信的孩子,万龙谷 龙族幻想手绘插画集,STEAM探索 法布尔昆虫记互动立体书 蜂和红蚂蚁
5,程序设计竞赛专题挑战教程,小型智能机器人制作全攻略 第5版,数学与创造：广中平祐自传,无线电监测站防雷与接地技术,拍出有温度的照片 零基础人像摄影课,中国数字经济政策全景图,全面预算管理与企业年度经营计划,习惯：捕捉95%的惯性思维，让用户对你的产品上瘾,关键演讲：每一次演讲都是人生的重要时刻,冒名顶替综合征,中国经典纹样图鉴,培生经典音乐教程 视唱练耳基础 第十版,故事策略 剧作必备的23个故事段落 全新修订版,NSCA-TSAC-F美国国家体能协会特种行业体能教练认证指南,司珍集 古风手工饰品定制技法完全图解,给孩子一个幸福的家,韵味东方 国潮插画绘制实例教学,STEAM探索 法布尔昆虫记互动立体书 蜣螂
6,图神经网络：基础、前沿与应用,怎样识别和检测电子元器件 第2版,星耀中国：我们的风云气象卫星（精装版）,5G承载网络运维 初级,无人机航拍摄影与摄像实战教程,企业会计准则详解与实务：条文解读+实务应用+案例讲解（2023年版）,简单做事,超常规思维：如何做出更明智的决策,情绪钝感力 不要什么都往心里去,情绪心理画 一笔一画安顿你的内心,荒墨 商业书法案例与应用实战,青少年美育趣味课堂 声乐基础,哈利波特电影魔法书 第2版,精准拉伸 疼痛消除和损伤预防的针对性练习 尊享版,绽放 缠花创意设计及手工制作指南,唤醒孩子的内在成长,悍将108,海底小纵队呼叫野外小纵队 第一辑
7,计算机体系结构：量化研究方法（第6版）,电子元器件识别与检测 全彩微视频版,微积分溯源：伟大思想的历程,通信简史,抖音博主实操攻略 内容策划视频制作直播技巧运营变现,中国制造：民族复兴的澎湃力量,新潮职业,一小时漫画缠论实战法,反本能：如何对抗你的习以为常,别对我说谎 如何发现别人隐藏的东西,人人都是设计师 视觉设计的原则和方法,青少年美育趣味课堂 音乐基础,电影人之眼 镜头的语言与表现力,腰部功能强化训练 预防损伤 缓解慢性疼痛与提升运动表现,打造理想水族箱 无须换水的鱼缸养护指南,面向未来的养育,镜花缘画谱,爱丽丝奇境解谜记


In [39]:
#【练习】
#利用selenium获取网页数据,并通过lxml获取网页数据--不重复，不漏爬
#获取新书推荐中，尝试获取计算机、电子、科普等若干类目的 8本图书

#本例用的wait.until(EC.visibility_of_all_elements_located((By.XPATH,...)) 形成列表再循环


from selenium import webdriver
from lxml import etree
import time

#导入相应的类模块及函数
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

driver = webdriver.Chrome()       # 打开chrome浏览器
driver.get('https://www.ptpress.com.cn/') #访问指定页面，主页

# 等待加载
time.sleep(3)
# 下拉滑块
driver.execute_script('scrollTo(0,3000)')
lista = driver.find_elements_by_xpath('//*[@id="newBook"]/div[2]/div[1]/span')
zda = {}
for i in range(len(lista)):
    lista[i].click()
    # 等待加载
    time.sleep(3)
    t1 = lista[i].text
    listc = driver.find_elements_by_xpath('//*[@id="newBook"]/div[2]/div[2]/div/div/a/p')
    listd = []
    for i in listc:
        listd.append(i.text)
    zda[t1] = listd
pd.DataFrame(zda)

,计算机,电子,科普,通信,摄影,经济,管理,金融与投资,成功/励志,心理学,设计,音乐,影视传媒,体育,生活,家教育儿,绘画,少儿
0,Python编程快速上手2：趣味小项目轻松学,运算放大器权威指南（第5版）,考试脑科学2：记忆、压力、动机的脑科学真相,5GtoB字典及面向深度融合的6G演进,故宫秘境,从零到亿,管理的博弈,股票大作手回忆录 丁圣元注疏版（签章版）,新潮职业,津巴多普通心理学（第8版）,商业插画设计思维魔方 形色构成+构图+色彩+光影+造型语言+插画后期+原创思维+综合训练,我的第一套音乐启蒙书 世界名曲篇,Hi-Fi High-End音响设计史,膝关节功能强化训练 预防损伤 缓解慢性疼痛与提升运动表现,新手养猫指导手册 从读懂猫咪心理到温暖相伴一生,日记十招：特级教师高子阳教你轻松写日记,花姬 Hanasa原子Dan双人典藏插画集,丝绸之路历史百科绘本 乘风破浪 开往世界的船
1,人工智能：现代方法（第4版）（上下册）,无线电合订本 66周年版上,视觉之旅：神奇的化学元素（少儿版）,5G+AI融合全景图,造像之术 职业修图师的商业摄影后期精修 实战篇,不公平优势：如何找到阻力最小的成功路径,激活组织能量：打造有机组织，创造非凡业绩,跨境电商亚马逊是如何运营的 从零做到年销千万实操手册,费曼学习法：我就是这样考上清华的,如何拥抱一只刺猬：恋爱与婚姻中的人格识别、接纳与付出,青少年美育趣味课堂 XMind思维导图制作,音乐制作自学手册 Cubase编曲案例分析与实战教学 视频版,什么是故事 给导演和编导的编剧与叙事指南,肩关节功能强化训练 预防损伤缓解慢性疼痛与提升运动表现,细工花饰的创意手作笔记 四时之趣,闯进诗词才子班 秒懂漫画古诗词 第一辑,动漫人体结构表现技法专项训练,丝绸之路历史百科绘本 勇往直前 走向世界的路
2,动手学深度学习（PyTorch版）,无线电合订本 66周年版下,量子矩阵：奇异的量子世界之旅,6G移动通信系统 理论与技术,一本书学会手机短视频拍摄,股权架构设计与股权激励,数智化敏捷组织：云钉一体驱动组织转型,金税四期管控下企业纳税筹划实务指南,你的灯亮着吗：如何找到真问题（10万册纪念版）,悲伤即成长：那些打不倒你的 只会让你更强大,扁平插画绘制专业教程,简谱约翰汤普森简易钢琴教程 五线谱+简谱版,混音指南 第2版,谢军少儿国际象棋启蒙绘本,手鞠球的几何世界 手鞠球制作教程,认知觉醒：伴随一生的学习方法论（青少年学习版）,鬼刀3 苍蓝 WLOP典藏插画作品集,STEAM探索 法布尔昆虫记互动立体书 蜘蛛和蝎子
3,人工智能伦理与治理——未来视角,模拟音频功率放大器设计,美丽的化学元素,御风前行 广电5G融合之路,故事感美食摄影,静水流深：高水平对外投资的实践逻辑,刷新管理：数字化企业快速成长,创新定价：世界知名企业的最大化盈利法则,复盘高手：自我认知与自我精进的底层逻辑,认知行为疗法：会谈技能与咨询现场,品牌设计法则,培生经典音乐教程 音乐理论基础 第三版,影视特效 运动匹配的艺术与技巧,运动营养全书 第5版,簪娘入门指南 古风饰品制作教程,费曼学习法（漫画版）：清华学霸的高效学习手册,鬼刀 WLOP个人插画作品集 典藏修订版,奇思妙想树叶画 一学就会的创意植物手工
4,RISC-V体系结构编程与实践,STM32入门100步,课堂上学不到的数学,自动驾驶电动汽车的计算与通信架构 车载网络通信架构,造像之术 职业修图师的商业摄影后期精修 技法篇,数字政府：开辟国家治理现代化新境界,华为的意志：华为经营逻辑的引擎,一小时漫画基金实战法,跑赢：成功不止一种定义,我会永远抱紧你,Procreate新国风插画绘制技法与商业应用,音乐制作自学手册 Logic Pro编曲案例分析与实战教学 视频版,拍摄手册：77种方法让你的影片更完美,抗阻训练方案设计指南 第4版,超可爱立体书 手工小机关制作入坑指南,适度养育：培养独立且自信的孩子,万龙谷 龙族幻想手绘插画集,STEAM探索 法布尔昆虫记互动立体书 蜂和红蚂蚁
5,程序设计竞赛专题挑战教程,小型智能机器人制作全攻略 第5版,数学与创造：广中平祐自传,无线电监测站防雷与接地技术,拍出有温度的照片 零基础人像摄影课,中国数字经济政策全景图,全面预算管理与企业年度经营计划,习惯：捕捉95%的惯性思维，让用户对你的产品上瘾,关键演讲：每一次演讲都是人生的重要时刻,冒名顶替综合征,中国经典纹样图鉴,培生经典音乐教程 视唱练耳基础 第十版,故事策略 剧作必备的23个故事段落 全新修订版,NSCA-TSAC-F美国国家体能协会特种行业体能教练认证指南,司珍集 古风手工饰品定制技法完全图解,给孩子一个幸福的家,韵味东方 国潮插画绘制实例教学,STEAM探索 法布尔昆虫记互动立体书 蜣螂
6,图神经网络：基础、前沿与应用,怎样识别和检测电子元器件 第2版,星耀中国：我们的风云气象卫星（精装版）,5G承载网络运维 初级,无人机航拍摄影与摄像实战教程,企业会计准则详解与实务：条文解读+实务应用+案例讲解（2023年版）,简单做事,超常规思维：如何做出更明智的决策,情绪钝感力 不要什么都往心里去,情绪心理画 一笔一画安顿你的内心,荒墨 商业书法案例与应用实战,青少年美育趣味课堂 声乐基础,哈利波特电影魔法书 第2版,精准拉伸 疼痛消除和损伤预防的针对性练习 尊享版,绽放 缠花创意设计及手工制作指南,唤醒孩子的内在成长,悍将108,海底小纵队呼叫野外小纵队 第一辑
7,计算机体系结构：量化研究方法（第6版）,电子元器件识别与检测 全彩微视频版,微积分溯源：伟大思想的历程,通信简史,抖音博主实操攻略 内容策划视频制作直播技巧运营变现,中国制造：民族复兴的澎湃力量,新潮职业,一小时漫画缠论实战法,反本能：如何对抗你的习以为常,别对我说谎 如何发现别人隐藏的东西,人人都是设计师 视觉设计的原则和方法,青少年美育趣味课堂 音乐基础,电影人之眼 镜头的语言与表现力,腰部功能强化训练 预防损伤 缓解慢性疼痛与提升运动表现,打造理想水族箱 无须换水的鱼缸养护指南,面向未来的养育,镜花缘画谱,爱丽丝奇境解谜记


In [4]:
#【练习】爬取当当畅销书本   （前10页 或 全部爬取）

#    和人邮存在的不同之处：人邮是异步加载，在右键-网页源码中查不到书本信息，但清华和当当的都是在源码中能查到，
#    验证：书籍信息是同步加载，不用sleep强制等待，但翻页按钮是异步加载的， 必须让滚动条滚到页面底部，翻页按钮出现，否则会出错
#    翻页时，如果只爬取前10页，则不必判断下一页按钮是否失效
#    如果是全部爬完，则需要注意，这一题翻页按钮a的上级按钮为 < li class='next'，但是到最后一页变为 <li class='next none'
#    所以，对a的定位不能用li，只能用a本身，而a本身没有特殊特征，可以用其含有文本“下一页”来做判断
  
#    本题未加sleep，可以正常爬取数据


import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from lxml import etree
import time

driver = webdriver.Chrome()       # 打开chrome浏览器
url='http://book.dangdang.com/'
driver.get(url) #访问 当当 主页

driver.maximize_window()#当前窗口最大化



,书名,作者,出版日期,出版社,现价,原价,评论数
0,你也走了很远的路吧（新增2万余字，4篇文章，关于特殊时期成长的...,卢思浩,2022-09-26,湖南文艺出版社,¥24.90,¥49.80,292868
1,我与地坛（纪念版）,史铁生,2011-06-01,人民文学出版社,¥19.40,¥29.00,424410
2,法治的细节（罗翔新作，法律随笔，评热点、论法理、聊读书、谈爱...,罗翔,2021-11-01,云南人民出版社,¥24.90,¥49.80,284113
3,真希望你也喜欢自己（房琪shou部作品重磅上市！写给所有年轻人的...,房琪kiki,2022-09-28,北京联合出版有限公司,¥37.30,¥49.80,419277
4,带壳的牡蛎是大人的心脏（当当专享签章版+作者亲绘贴纸。火爆全网...,拟泥nini,2022-09-01,北京联合出版有限公司,¥40.80,¥68.00,136824
...,...,...,...,...,...,...,...
495,影响力（全新升级版）2021新版 “影响力教父”罗伯特·西奥迪尼新...,罗伯特·西奥迪尼,2021-11-06,北京联合出版有限公司,¥74.90,¥99.90,336161
496,围城 钱锺书著（2022年精装新版）,钱锺书,2022-07-01,人民文学出版社,¥36.20,¥48.00,1231065
497,工作、消费主义和新穷人,齐格蒙特·鲍曼,2021-09-01,上海社会科学院出版社,¥36.20,¥45.00,21886
498,大侦探福尔摩斯（第9辑）（全4册）,厉河,2019-03-01,湖北教育出版社,¥60.00,¥76.00,46887


In [51]:
#【练习】爬取当当畅销书本   （前10页 或 全部爬取）

#    和人邮存在的不同之处：人邮是异步加载，在右键-网页源码中查不到书本信息，但清华和当当的都是在源码中能查到，
#    验证：书籍信息是同步加载，不用sleep强制等待，但翻页按钮是异步加载的， 必须让滚动条滚到页面底部，翻页按钮出现，否则会出错
#    翻页时，如果只爬取前10页，则不必判断下一页按钮是否失效
#    如果是全部爬完，则需要注意，这一题翻页按钮a的上级按钮为 < li class='next'，但是到最后一页变为 <li class='next none'
#    所以，对a的定位不能用li，只能用a本身，而a本身没有特殊特征，可以用其含有文本“下一页”来做判断
  
#    本题未加sleep，可以正常爬取数据


import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from lxml import etree
import time

driver = webdriver.Chrome()       # 打开chrome浏览器
url='http://book.dangdang.com/'
driver.get(url) #访问 当当 主页

driver.maximize_window()#当前窗口最大化

# 睡眠3秒
time.sleep(3)
# 下拉滑块
driver.execute_script('scrollTo(0,1000)')

# 点击热销榜
driver.find_element_by_xpath('//*[@id="bd_auto"]/div[3]/div[3]/div[1]/div[2]/a').click()

# 重定位
driver.switch_to.window(driver.window_handles[-1])

# 睡眠3秒
time.sleep(3)
lista = ['书名', '作者', '出版日期', '出版社','现价','原价','评论数']
list1 = []
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
# /html/body/div[3]/div[3]/div[2]/ul/li[1]/div[5]/a[1]
while True:
    # 睡眠3秒
    time.sleep(3)
    listb = driver.find_elements_by_xpath('/html/body/div[3]/div[3]/div[2]/ul/li')
    for i in range(len(listb)):
        # 书名
        t1 =listb[i].find_element_by_xpath('./div[3]/a').get_attribute('title')
        list1.append(t1)
        # 作者
        t2 = listb[i].find_element_by_xpath('./div[5]/a[1]').text
        list2.append(t2)
        # 出版日期/div[6]/span
        t3 = listb[i].find_element_by_xpath('./div[6]/span').text
        list3.append(t3)
        # 出版社/div[6]/a
        t4 = listb[i].find_element_by_xpath('./div[6]/a').text
        list4.append(t4)
        # 现价/div[7]/p[1]/span[1]
        t5 = listb[i].find_element_by_xpath('./div[7]/p[1]/span[1]').text
        list5.append(t5)
        # 原价/div[7]/p[1]/span[2]
        t6 = listb[i].find_element_by_xpath('./div[7]/p[1]/span[2]').text
        list6.append(t6)
        # 评论数/div[4]/a
        t7 = listb[i].find_element_by_xpath('./div[4]/a').text
        list7.append(t7[:-3])
        
    x = driver.find_element_by_css_selector('.next')
    y = x.get_attribute('class')
    if 'none' in y:
        break
    else:
        x.click()


In [52]:
zda = {}

In [53]:
zda[lista[0]] =list1
zda[lista[1]] =list2
zda[lista[2]] =list3
zda[lista[3]] =list4
zda[lista[4]] =list5
zda[lista[5]] =list6
zda[lista[6]] =list7

In [54]:
zda

{'书名': ['你也走了很远的路吧（新增2万余字，4篇文章，关于特殊时期成长的勇气，以及曾经的那些人）',
  '我与地坛（纪念版）',
  '法治的细节（罗翔新作，法律随笔，评热点、论法理、聊读书、谈爱情，人间清醒与你坦诚相见）',
  '真希望你也喜欢自己（房琪shou部作品重磅上市！写给所有年轻人的成长通关秘籍！杨天真、大冰、鲁豫、张含韵诚挚作序推荐。）',
  '带壳的牡蛎是大人的心脏（当当专享签章版+作者亲绘贴纸。火爆全网现象级漫画，白茶、沈月、阮筠庭、文森特动物园推荐！）',
  '被讨厌的勇气：“自我启发之父”阿德勒的哲学课 岸见一郎',
  '蛤蟆先生去看心理医生（热销300万册！英国经典心理咨询入门书，知名心理学家李松蔚强烈推荐）',
  '生死疲劳（不看不知道，莫言真幽默！全新版本！印章版和普通版随机发货）',
  '钝感力（渡边淳一经典励志大作！央视新闻、《奇葩说》鼎力推荐，马东、蔡康永、杨天真、王俊凯推荐书目。写给易因小事敏感，什么都往心里去的人。)',
  '活着（余华代表作，精装，易烊千玺推荐阅读）',
  '病隙碎笔 2021纪念版（史铁生充满灵性光辉的生命笔记，启迪无数读者的长篇哲思散文经典）',
  '长安的荔枝',
  '三体：全三册 刘慈欣代表作，亚洲“雨果奖”获奖作品！',
  '第七天（2022版，余华长篇小说经典，比《活着》更绝望，比《兄弟》更荒诞，获华语文学传媒大奖）',
  '人生（茅盾文学奖得主路遥作品，电视剧《人生之路》原著，陈晓、李沁、张嘉倪主演。高加林一代青年冲开困境，去往人生新“路”。）',
  '我从未如此眷恋人间：周深“终于开始学会眷恋这人间”史铁生、季羡林、余光中、丰子恺等联手献作，把深情写入文字，告诉你这世间原来是它们最惹人恋。',
  '作家榜名著：月亮与六便士（全新未删节插图珍藏版！毛姆写给年轻人的梦想之书！满地都是六便士，他却抬头看见了月亮！）',
  '人生海海（麦家重磅力作，莫言、董卿盛赞，连续两年高居畅销榜，发行量超200万册）',
  '始于极限：女性主义往复书简（上野千鹤子新作，万千读者无限共鸣的坦诚对话）',
  '翦商：殷周之变与华夏新生（许宏作序；罗翔年度推荐；《南方周末》2022年度十大好书；一部夏商周启示录）',
  '圆圈正义',
  '瓦尔登湖（经典新读 中央编译出版社

In [55]:
pd.DataFrame(zda)

,书名,作者,出版日期,出版社,现价,原价,评论数
0,你也走了很远的路吧（新增2万余字，4篇文章，关于特殊时期成长的勇气，以及曾经的那些人）,卢思浩,2022-09-26,湖南文艺出版社,¥24.90,¥49.80,295641
1,我与地坛（纪念版）,史铁生,2011-06-01,人民文学出版社,¥16.00,¥29.00,426987
2,法治的细节（罗翔新作，法律随笔，评热点、论法理、聊读书、谈爱情，人间清醒与你坦诚相见）,罗翔,2021-11-01,云南人民出版社,¥24.90,¥49.80,286475
3,真希望你也喜欢自己（房琪shou部作品重磅上市！写给所有年轻人的成长通关秘籍！杨天真、大冰、...,房琪kiki,2022-09-28,北京联合出版有限公司,¥37.30,¥49.80,420565
4,带壳的牡蛎是大人的心脏（当当专享签章版+作者亲绘贴纸。火爆全网现象级漫画，白茶、沈月、阮筠庭...,拟泥nini,2022-09-01,北京联合出版有限公司,¥40.80,¥68.00,140239
...,...,...,...,...,...,...,...
495,快乐就是哈哈哈哈哈,梁实秋,2022-08-01,中国友谊出版公司,¥26.00,¥52.00,15126
496,【全6册】写给孩子的山海经 儿童中小学生无障碍彩图注音有声伴读三四五年级课外阅读书籍3-6-...,张佳杰,2022-04-01,中国国际广播出版社,¥90.00,¥158.00,2948
497,给孩子的安全教育漫画：别想欺负我,孩子的安全官,2022-05-31,北京联合出版有限公司,¥27.30,¥38.00,16294
498,丰子恺万般滋味，都是生活,丰子恺,2018-09-01,华中科技大学出版社,¥21.00,¥42.00,645186
